In [32]:
import rasterio
from rasterio.plot import show
import json
import pandas as pd
import numpy as np

In [5]:
class DensityExtractor():
    def __init__(self, data_path: str):
        self._path_to_csv_data = data_path

    def get_csv_data(self) -> pd.DataFrame:
        csv_data = pd.read_csv(self._path_to_csv_data)
        csv_data = csv_data.rename(
            columns=
            {
                "X": "longitude",
                "Y": "latitude",
                "Z": "population_density"
            }
        )
        return csv_data

In [6]:
directory_density_path = "../config/cfg_density_population.json"

with open(directory_density_path) as f:
    data_json = json.load(f)

# Print the data (it will be stored as a Python dictionary)
density_population_path = data_json['path_csv']

dens_extractor = DensityExtractor(density_population_path)

In [8]:
df = dens_extractor.get_csv_data()
df

,longitude,latitude,population_density
0,30.436250,-1.046250,358.407013
1,30.444583,-1.046250,610.205383
2,30.452917,-1.046250,1437.951172
3,30.461250,-1.046250,2122.332031
4,30.402917,-1.054583,187.413574
...,...,...,...
28399,29.377917,-2.821250,0.000000
28400,29.519583,-2.821250,414.417999
28401,29.527917,-2.821250,409.061432
28402,29.536250,-2.821250,416.264374


In [44]:
# print(df['longitude'].dtype)
print(df['longitude'].round(6) - 30.436250)

0        0.000000
1        0.008333
2        0.016667
3        0.025000
4       -0.033333
           ...   
28399   -1.058333
28400   -0.916667
28401   -0.908333
28402   -0.900000
28403   -1.066667
Name: longitude, Length: 28404, dtype: float64


In [45]:
df_prova = df[np.isclose(df['longitude'],30.436250)] # funziona
df_prova = df[df['longitude'].round(6) == 30.436250] # funziona
df_prova

,longitude,latitude,population_density
0,30.43625,-1.046250,358.407013
7,30.43625,-1.054583,370.727692
23,30.43625,-1.062916,346.225922
38,30.43625,-1.071250,278.492615
52,30.43625,-1.079583,292.766083
...,...,...,...
21000,30.43625,-2.287916,242.237976
21210,30.43625,-2.296250,264.536957
21425,30.43625,-2.304583,279.894257
21637,30.43625,-2.312916,276.680603
